# Mathematical foundations of CL

Selective disclosure and the AnonCred format requires a signature scheme that supports signing multiple messages with a single signature. The Camenisch-Lysyanskaya (CL) signature scheme is a multi message capable signature scheme that was developed in a series of papers ([CL01](https://eprint.iacr.org/2001/019.pdf), [CL02](https://cs.brown.edu/people/alysyans/papers/camlys02.pdf), [CL03](http://cs.brown.edu/people/alysyans/papers/camlys02b.pdf), [CL04](https://www.iacr.org/archive/crypto2004/31520055/cl04.pdf)). To understand CL signatures, it is important to understand both the Discrete Log Problem and Cryptographic Commitments.

## Discrete Log Problem

Given a multiplicative cyclic group $G$ a generator $g$ can generate every element $h \in G$ as $g^x$ for some $x$. The **discrete log** to the base $g$ of $h$ in the group $G$ is defined as $x$.

Example: For $G = \mathbf{Z}_5$, and $g=2$, the discrete log of 1 is 4 because $2^4 \equiv 1 \pmod 5$

The **discrete log problem** is defined as: given $(G,g,h)$, find the base $g$ discrete log of $h$ in $G$. Note that the discrete log problem is not always hard. The hardness depends on the choice of the groups. A prime order group $\mathbf{Z}_p$ where $p-1$ is not a product of small primes (it is common to use a [safe prime](https://en.wikipedia.org/wiki/Safe_and_Sophie_Germain_primes#:~:text=The%20number%202p%20%2B%201%20associated,is%20its%20associated%20safe%20prime.&text=Sophie%20Germain%20primes%20and%20safe,key%20cryptography%20and%20primality%20testing.) of at least 1024 bits) is safe in the sense that we know of no efficient algorithms for solving the discrete log in that group.

## Cryptographic commitments

A commitment scheme is a protocol that allows one to commit a message publicly with the option to reveal the message when so desired. The [Pedersen commitment scheme](https://web.archive.org/web/20170811001441/https://eprint.iacr.org/2004/201.pdf), in its simplified version, is a commitment $c$ of the message $m$ as $c \equiv a^m \pmod n$ where $a$ is a member of a large multiplicate cyclic group $G$. Due to the discrete log problem, a receiver given $c$ cannot find $m$, but it is trivially easy to verify the commitment if the committer shares $m$.

## CL signatures

### Strong RSA based 

Originally, CL signatures were based on the strong RSA assumption. This resulted in large key sizes to provide sufficient security against prime factorization. The actors and their interactions are as follows:

* An issuer signs a signature on a block of messages
* The holder presents the messages, or a subset thereof, to a verfier together with the signature
* The verifier checks the messages and the signature

The CL signature scheme consists of three steps ([source](https://www.csc.kth.se/~buc/PPC/Slides/jan.pdf)):

1. Key generation
2. Signature generation
3. Signature verification

**Key generation**. 

Prerequisites: a collision-free hash function $H:\{0,1\}^{*} \rightarrow \{0,1\}^{l}$ 

* Private key: two random primes $(p,q)$
* Public key: RSA modulus $n = pq$, and $a_i,b,d \in \mathbf{QR}_n$ (ie., generates $K+2$ random numbers from a [quadratic residue](https://en.wikipedia.org/wiki/Quadratic_residue) modulo $n$)

**Signature generation**

* The issuer generates a random prime $e > 2^l$ and a random number $s \approx n$. 
* Compute $c$ s.t. 

\begin{align}
    d \equiv a_1^{m_1} \cdot \ldots \cdot a_k^{m_k} b^s c ^e \pmod n
\end{align}

The signature is $(c,e,s)$.

**Signature verification** 

Given a signature $(c,e,s)$ of messages $\{m_1,\ldots,m_k\}$, a verifier can verify the signature using the issuer's public key.

## Selective disclosure with CL signatures

The holder can selectively disclose a message by:

1. sending $m_i$ to the verifier, who computes $(a_i^{m_i}) \pmod n$
2. compute $(a_i^{m_i}) \pmod n$, which is the commitment of $m_i$ and sent the commitment to he verifier

The holder can send over the messages they wish to disclose, and the commitment for the messages they wish to keep secret.

### The alternative as shown on [medium](https://medium.com/finema/anonymous-credential-part-2-selective-disclosure-and-cl-signature-b904a93a1565)

The steps are the same as above, but the medium link provides the computation as follows:

\begin{align}
    v^e \equiv a_1^{m_1} \cdot \ldots \cdot a_k^{m_k} b^s c \pmod n
\end{align}

This is the same as the one above with the step of computing the $e^{-1}$, using $(p-1)(q-1)$ as modulus, and raising both the RH and LS to it.

In [1]:
#!conda install -c conda-forge pycryptodome -y # for docker jupyter labs
from Crypto.Util import number

In [2]:
# The medium version
p = number.getPrime(32) # We ignore safe primes
q = number.getPrime(32) # We ignore safe primes
n = p*q

a_1 = pow(number.getRandomRange(2, n), 2, n)
a_2 = pow(number.getRandomRange(2, n), 2, n)
b = pow(number.getRandomRange(2, n), 2, n)
c = pow(number.getRandomRange(2, n), 2, n)

m_1 = number.getRandomRange(2, n)
m_2 = number.getRandomRange(2, n)

e = number.getPrime(m_1.bit_length() + 1)
e_inv = pow(e, -1, (p-1)*(q-1))
s = number.getRandomNBitInteger(m_1.bit_length() + n.bit_length() + 1)

RH = (pow(a_1, m_1, n) * pow(a_2, m_2, n) * pow(b, s, n) * c) % n
v = pow(RH, e_inv, n)

print(f'Public key is: (a_1, a_2) = {(a_1, a_2)}, b = {b}, c = {c}, n = {n}')
print(f'Private key is:', (p,q))
print(f'\nSignature on m_1, m_2 ({m_1}, {m_2}) is: e = {e}, s = {s}, v = {v}')

print('RHS is same as LHS:', pow(v, e, n) == (pow(a_1, m_1, n) * pow(a_2, m_2, n) * pow(b, s, n) * c) % n)

Public key is: (a_1, a_2) = (550828409666538705, 9571406797102341698), b = 2600824706072779672, c = 5866881841088568039, n = 11149101156031745201
Private key is: (2689281043, 4145755307)

Signature on m_1, m_2 (10674260871497813288, 3754177875595527393) is: e = 28284475931224289779, s = 484321854995968823211740647871723565820, v = 2475067959638636802
RHS is same as LHS: True


### The traditional way as shown in CL01

Steps:

* Generate $n=pq$
* $R_0, \ldots, R_{L-1}, S, Z \in \mathbf{QR}_n$
* Public key will be $(n,R_0, \ldots, R_{L-1}, S, Z)$
* Secret key is $p$
* Message space is $\{(m_0, \ldots, m_{L-1}): m_i \in \pm \{0,1\}^{l_m}\}$
* Signing algorithm: on input $(m_0, \ldots, m_{L-1})$ chose a random prime $e>l_m + 2$ and a random number $v$ of length $l_v = l_n + l_m + l_r$ where $l_r$ is a security parameter
* Compute the value $A$ s.t. 

\begin{align}
  A = \Bigg(\frac{Z}{R_0^{m_0} \cdot \ldots \cdot R_{L-1}^{m_{L-1}} \cdot S^v}\Bigg)^{\frac{1}{e}} \pmod n
\end{align}

* The signature on the message $(m_0, \ldots, m_{L-1})$ consists of $(e,A,v)$

Verification of the signature on the message is done through:

* $Z \equiv A^e \cdot R_0^{m_0} \cdot \ldots \cdot R_{L-1}^{m_{L-1}} \cdot S^v \pmod n$

In [3]:
p = number.getPrime(32) # We ignore safe primes
q = number.getPrime(32) # We ignore safe primes
n = p*q

R_0 = pow(number.getRandomRange(2, n), 2, n)
R_1 = pow(number.getRandomRange(2, n), 2, n)
S = pow(number.getRandomRange(2, n), 2, n)
Z = pow(number.getRandomRange(2, n), 2, n)

m_0 = number.getRandomRange(2, n)
m_1 = number.getRandomRange(2, n)

e = number.getPrime(m_0.bit_length() + 1)
e_inv = pow(e, -1, (p-1)*(q-1))
v = number.getRandomNBitInteger(m_0.bit_length() + n.bit_length() + 1)

X = pow(R_0, m_0, n) * pow(R_1, m_1, n) * pow(S, v, n)
X_inv = pow(X, -1, n)
A = pow((Z * X_inv) % n, e_inv, n)

print(f'Public key is: n = {n}, (R_0, R_1) = {(R_0, R_1)}, S = {S}, Z = {Z}')
print(f'Private key is:', (p,q))
print(f'\nSignature on m_0, m_1 ({m_0}, {m_1}) is: e = {e}, s = {A}, v = {v}')

print('RHS is the same as LHS:', (pow(A, e, n) * X) % n == Z)

Public key is: n = 9171217927405769009, (R_0, R_1) = (8192245720955003556, 6667589631636168374), S = 2735196351304324892, Z = 4847505334916751160
Private key is: (2722736041, 3368383049)

Signature on m_0, m_1 (1181652844703113230, 1205699119803473053) is: e = 4210433832471942679, s = 3369317229320912901, v = 26954893583704235995610919337064775128
RHS is the same as LHS: True
